In [1]:
import tifffile
try:
    import napari
except ModuleNotFoundError:
    pass
import os
import numpy as np
import platform
import time
try:
    from skimage.morphology import isotropic_dilation, isotropic_erosion
    import stackview
except ImportError:
    pass
import matplotlib.pyplot as plt

The first step is to extract smaller patches which we later refine. 
(This work flow was originally necessary as the data was stored on a cluster and napari, which is used to annotate the data, was not able to run remotely. The small patches were therefore easier to analyse and load into napari locally.)

In [4]:
# Needs to be filled in!
nhs_channel = 1
path_nhs = ''    # Path for NHS
path_inner = ''  # Path for predictions of initial inner structure based on HeLa model
path_outer = ''  # Path for predictions of the outline

# If we only refine predictions, use this path
full_prediction = False
path_full_pred = '' # Path for initial predictions that we want to refine

# Potential dilation for the outline
dilation = -14

# List of patches to annotate
location = [
    ('WT2_60X_0.tif', (60, 1250, 750)),   # File, (z, y, x)
]

depth = 32
width = 512


for file, (z,y,x) in location:
    print(file)

    nhs = tifffile.imread(os.path.join(path_nhs, file))[:,nhs_channel]
    qmin, qmax = np.quantile(nhs.flatten(), q = [0.0025, 0.9985])  # Thresholds might have to be adopted for new data
    nhs = np.clip(nhs, qmin, qmax)
    nhs = (nhs - qmin) / (qmax - qmin)

    n = nhs[z:z+depth, y:y+width, x:x+width]
    n = (n * 255).astype('uint8')
    
    if not full_prediction:
        inner = tifffile.imread(os.path.join(path_inner, file))
        outer = tifffile.imread(os.path.join(path_outer, file.replace(".tif", "pred_bin.tif"))) 
        p = inner[z:z+depth, y:y+width, x:x+width].copy()
        o = outer[z:z+depth, y:y+width, x:x+width].copy()
        if dilation > 0:
            o = np.stack([isotropic_dilation(s, dilation) for s in o], axis=0)
        if dilation < 0:
            o = np.stack([isotropic_erosion(s, -dilation) for s in o], axis=0)
        p[o==0]= 0  
    else:
        p = tifffile.imread(os.path.join(path_full_pred, file))
        p = p[z:z+depth, y:y+width, x:x+width].copy()

    file_name = file.replace('.tif', f'_nhs_{z}_{y}_{x}.tif')
    pred_name = file.replace('.tif', f'_pred_{z}_{y}_{x}.tif')
    tifffile.imwrite(f'data/{file_name}', 
        n,
        imagej=True, 
        metadata={'axes': 'ZYX'}, 
        compression ='zlib')
    tifffile.imwrite(f'data/{pred_name}', 
        p,
        imagej=True, 
        metadata={'axes': 'ZYX'}, 
        compression ='zlib')


# stackview.side_by_side(n, p.astype('int64'))

WT2_60X_7.tif
WT2_60X_8.tif
WT2_60X_8.tif
WT2_60X_9.tif


The following code is now supposed to be executed on a machine which can run napari.

In [2]:

class Files:
    def __init__(self, base_path, refine=False, ambiguous = False, prefix=''):
        self.base_path = base_path
        self.files =  [f for f in os.listdir(base_path) if f.endswith('.tif') and not 'pred' in f]
        if prefix:
            self.files = [f for f in self.files if prefix in f]
        self.refine = refine
        self.ambiguous = ambiguous
        if ambiguous:
            print(f'Attention!!! Ambiguous is turned on!')

    def get_file(self):
        if not self.refine:
            while self.files:
                file = self.files.pop()
                path = os.path.join(self.base_path, file)
                pred_file =  path.replace('nhs', 'pred')
                self.target_file = pred_file.replace('pred', 'pred_mod')
                if self.ambiguous:
                    pred_file =  path.replace('nhs', 'pred_mod')
                    self.target_file = self.target_file.replace('mod', 'mod_ambig')
                print(self.target_file)
                if not os.path.exists(self.target_file):
                    break
        else:
            while self.files:
                file = self.files.pop()
                path = os.path.join(self.base_path, file)
                self.target_file = path.replace('nhs', 'pred_mod')
                if self.ambiguous:
                    pred_file =  path.replace('nhs', 'pred_mod')
                    self.target_file = self.target_file.replace('mod', 'mod_ambig')
                if os.path.exists(self.target_file):
                    break
        print(f'Annotating: {file}')
        nhs = tifffile.imread(path)
        pred = tifffile.imread(pred_file)
        return nhs, pred

# files = Files(base_path, refine=False)
base_path = 'path/InnerStructure/data'
files = Files(base_path, refine=False, ambiguous=False, prefix='WT2')


In [7]:
nhs, pred = files.get_file()

viewer = napari.Viewer()
viewer.add_image(nhs, name='nhs')
viewer.add_labels(pred, name='pred', opacity=0.3)
viewer.layers['pred'].brush_size = 9

IGNORE_VALUE = 6

@viewer.bind_key('r')
def toggle_preds(viewer):
    viewer.layers['pred'].visible = not viewer.layers['pred'].visible

    if viewer.layers['pred'].visible:
        viewer.layers['pred'].mode = 'paint'

@viewer.bind_key('q')
def dec_brush(viewer):
    viewer.layers['pred'].brush_size = max(1, viewer.layers['pred'].brush_size - 1)

@viewer.bind_key('w')
def inc_brush(viewer):
    viewer.layers['pred'].brush_size = min(30, viewer.layers['pred'].brush_size + 1)

@viewer.bind_key('e')
def toggle_preds(viewer):
    if viewer.layers['pred'].mode == 'paint':
        viewer.layers['pred'].mode = 'pan_zoom'
    if viewer.layers['pred'].mode == 'pan_zoom':
        viewer.layers['pred'].mode = 'paint'

@viewer.bind_key('c')
def select_cristae(viewer):
    viewer.layers['pred'].selected_label = 2

@viewer.bind_key('i')
def select_IM(viewer):
    viewer.layers['pred'].selected_label = 1

@viewer.bind_key('b')
def select_BG(viewer):
    viewer.layers['pred'].selected_label = 0

@viewer.bind_key('m')
def select_matrix(viewer):
    viewer.layers['pred'].selected_label = 3

@viewer.bind_key('p')
def select_IG(viewer):
    viewer.layers['pred'].selected_label = 4

@viewer.bind_key('o')
def delete_pred_slice(viewer):
    z = int(viewer.dims.point[0])
    viewer.layers['pred'].data[z] = IGNORE_VALUE
    viewer.layers['pred'].refresh()


@viewer.bind_key('a')
def change_contour(viewer):
    if viewer.layers['pred'].contour == 0:
        viewer.layers['pred'].contour = 1
        viewer.layers['pred'].opacity = 0.8
    else:
        viewer.layers['pred'].contour = 0
        viewer.layers['pred'].opacity = 0.3

@viewer.bind_key('s')
def save_mod(viewer):
    mod_pred = viewer.layers['pred'].data.copy()
    tifffile.imwrite(files.target_file,
            mod_pred.astype('uint8'),
            metadata={'axes': 'ZYX'}, 
            compression ='zlib')
    print(time.strftime("%H:%M:%S"), 'Saved!')




/Users/alexandersauer/mnt_rescomp/AntiBodySegKidney/MitoClustering/data/WT2_60X_5_pred_mod_150_1500_800.tif
/Users/alexandersauer/mnt_rescomp/AntiBodySegKidney/MitoClustering/data/WT2_60X_3_pred_mod_120_1000_800.tif
Annotating: WT2_60X_3_nhs_120_1000_800.tif
